# Create PCAPs

In [1]:
from scapy.all import Ether, Dot1Q, RawPcapWriter, IP, UDP
from dpkt.pcap import Reader
from json import load
from pandas import DataFrame, to_numeric
from math import ceil, modf
from decimal import Decimal

## BigBuckBunny
- read audio frames as json and parse to pandas DataFrame

In [2]:
with open("BigBuckBunny/bbb_audio_frames.json") as audio_json_raw:
    audio_json = load(audio_json_raw)
df_a = DataFrame.from_dict(audio_json['frames'])
df_a['pkt_size'] = to_numeric(df_a['pkt_size'])

In [3]:
with open("BigBuckBunny/bbb_video_frames.json") as video_json_raw:
    video_json = load(video_json_raw)
df_v = DataFrame.from_dict(video_json['frames'])
df_v['pkt_size'] = to_numeric(df_v['pkt_size'])

- create pcaps from DataFrames

In [4]:
PATH = "pcap/"
ETH_DST = '1c:1b:0d:9a:70:8c'
VLAN_TYPE = 0x0800
VLAN_ID = 3

In [45]:
def get_header(payload, pkt_id, vlan_prio, eth_src):
    return Ether(dst=ETH_DST,
                     src=eth_src
                    )/Dot1Q(prio=vlan_prio,
                            vlan=VLAN_ID,
                            type=VLAN_TYPE
                           )/IP(id=pkt_id,
                                version=4,
                                dst='10.0.0.2',
                                src='10.0.0.1')/UDP(sport=10001,
                                                    dport=10002)/payload


def inc_pkt_id(pkt_id):
    if pkt_id >= 65535:
        return 0
    else:
        return pkt_id + 1

### audio pcap

In [8]:
AUDIO_PATH = PATH + 'audio.pcap'
AUDIO_ETH_SRC = '00:00:00:00:00:02'
AUDIO_PAYLOAD = 480 * '0'
AUDIO_VLAN_PRIO = 5

audio_pcap = RawPcapWriter(open(AUDIO_PATH, 'wb'), nano=False, linktype=1, sync=True, append=True)

for index, row in df_a.iterrows():
    time = row['best_effort_timestamp_time'].rsplit(".")
    sec = int(time[0])
    usec = int(time[1])
    audio_header = get_header(payload=AUDIO_PAYLOAD, pkt_id=index, vlan_prio=AUDIO_VLAN_PRIO, eth_src=AUDIO_ETH_SRC)
    audio_pcap.write_header(audio_header)
    audio_pcap.write_packet(audio_header, sec=sec, usec=usec)

### video pcap

In [42]:
VIDEO_PATH = PATH + 'video.pcap'
VIDEO_ETH_SRC = '00:00:00:00:00:03'
VIDEO_VLAN_PRIO = 4
FULL_PAYLOAD = 1472
FULL_PAYLOAD_ZEROS = FULL_PAYLOAD * '0'

video_pcap = RawPcapWriter(open(VIDEO_PATH, 'wb'), nano=True, linktype=1, sync=True, append=True)

In [43]:
pkt_id = 0
for _, row in df_v.iterrows():
    time = row['best_effort_timestamp_time'].rsplit(".")
    sec = int(time[0])
    nsec = int(time[1]) * 1000
    size = row['pkt_size']
    full, part = divmod(size, FULL_PAYLOAD)
    for _ in range(0, full):
        video_header = get_header(payload=FULL_PAYLOAD_ZEROS, pkt_id=pkt_id, vlan_prio=VIDEO_VLAN_PRIO, eth_src=VIDEO_ETH_SRC)
        video_pcap.write_header(video_header)
        video_pcap.write_packet(video_header, sec=sec, usec=nsec)
        nsec += 1
        pkt_id = inc_pkt_id(pkt_id)
    part = part * '0'
    video_header = get_header(payload=part, pkt_id=pkt_id, vlan_prio=VIDEO_VLAN_PRIO, eth_src=VIDEO_ETH_SRC)
    video_pcap.write_header(video_header)
    video_pcap.write_packet(video_header, sec=sec, usec=nsec)
    pkt_id = inc_pkt_id(pkt_id)

### robotic pcap

In [58]:
ROBOTIC_LENGTH = ceil(max(float(df_a.iloc[-1]['best_effort_timestamp_time']), float(df_v.iloc[-1]['best_effort_timestamp_time'])))

ROBOTIC_PATH = PATH + 'robotic.pcap'
ROBOTIC_ETH_SRC = '00:00:00:00:00:01'
ROBOTIC_VLAN_PRIO = 6
ROBOTIC_PAYLOAD = 82 * '0'

robotic_pcap = RawPcapWriter(open(ROBOTIC_PATH, 'wb'), nano=False, linktype=1, sync=True, append=True)

In [59]:
pkt_id = 0

for sec in range(0, ROBOTIC_LENGTH):
    for msec in range(0,1000):
        usec = msec * 1000
        robotic_header = get_header(payload=ROBOTIC_PAYLOAD, pkt_id=pkt_id, vlan_prio=ROBOTIC_VLAN_PRIO, eth_src=ROBOTIC_ETH_SRC)
        robotic_pcap.write_header(robotic_header)
        robotic_pcap.write_packet(robotic_header, sec=sec, usec=usec)
        pkt_id = inc_pkt_id(pkt_id)

## Haptic Glove

In [ ]:
def get_header(payload, pkt_id, vlan_prio, eth_dst, eth_src):
    return Ether(dst=eth_dst,
                 src=eth_src
                 )/Dot1Q(prio=vlan_prio,
                         vlan=VLAN_ID,
                         type=VLAN_TYPE
                         )/IP(id=pkt_id,
                              version=4,
                              dst='10.0.0.2',
                              src='10.0.0.1')/UDP(sport=10001,
                                                  dport=10002)/payload


def inc_pkt_id(pkt_id):
    if pkt_id >= 65535:
        return 0
    else:
        return pkt_id + 1

In [ ]:
PATH = "./"
ETH_DST = '1c:1b:0d:9a:70:8c'
VLAN_TYPE = 0x0800
VLAN_ID = 3

LEN_ETHERNET_HEADER = 18
LEN_IP_HEADER = 20
LEN_UDP_HEADER = 8
LEN_HEADER = LEN_ETHERNET_HEADER + LEN_IP_HEADER + LEN_UDP_HEADER
LEN_FRAME_MIN = 64

TACTILE_PATH_HOST = "{}tactile{}.pcap".format(PATH, "_host")
TACTILE_PATH_GLOVE = "{}tactile{}.pcap".format(PATH, "_glove")
TACTILE_ETH_SRC = "00:00:00:00:00:01"
TACTILE_VLAN_PRIO = 6

pcap_host = Reader(open("raw/senseglove_host_to_glove.pcap", "rb"))
pcap_glove = Reader(open("raw/senseglove_glove_to_host.pcap", "rb"))
pcaps = [pcap_host, pcap_glove]
tactile_pcap_host = RawPcapWriter(open(TACTILE_PATH_HOST, 'wb'), nano=False, linktype=1, sync=True, append=True)
tactile_pcap_glove = RawPcapWriter(open(TACTILE_PATH_GLOVE, 'wb'), nano=False, linktype=1, sync=True, append=True)
pkt_id_host = 0
pkt_id_glove = 0
TIME_REF_HOST = None
TIME_REF_GLOVE = None

for pcap in pcaps:
    for ts, buf in pcap:
        # first 27B = USB header, use payload as bytestream:
        hx = buf.hex()
        payload = bytes.fromhex(hx[54:])
        if len(payload) < (LEN_FRAME_MIN - LEN_HEADER):
            payload += str.encode('0' * (LEN_FRAME_MIN - LEN_HEADER)) # pad with zero with packet is too small

        if pcap == pcap_glove:
            try:
                # set time reference:
                if not TIME_REF_GLOVE:
                    TIME_REF_GLOVE = ts
                # create and write header for Ethernet frame:
                tactile_header = get_header(payload=payload, pkt_id=pkt_id_glove, vlan_prio=TACTILE_VLAN_PRIO, eth_dst=ETH_DST, eth_src=TACTILE_ETH_SRC)
                tactile_pcap_glove.write_header(tactile_header)

                # set correct time:
                ts_new = ts - TIME_REF_GLOVE
                usec, sec = modf(ts_new)
                sec = int(sec)
                usec = round(usec * 1e6)

                # write payload to Ethernet frame:
                tactile_pcap_glove.write_packet(tactile_header, sec=sec, usec=usec)
                pkt_id_glove = inc_pkt_id(pkt_id_glove)
            except Exception as e:
                print(e)
                print(ts, sec, usec, hx)

        elif pcap == pcap_host:
            # set time reference:
            if not TIME_REF_HOST:
                TIME_REF_HOST = ts
            # create and write header for Ethernet frame:
            tactile_header = get_header(payload=payload, pkt_id=pkt_id_host, vlan_prio=TACTILE_VLAN_PRIO, eth_dst=TACTILE_ETH_SRC, eth_src=ETH_DST)
            tactile_pcap_host.write_header(tactile_header)

            # set correct time:
            ts_new = ts - TIME_REF_HOST
            usec, sec = modf(ts_new)
            sec = int(sec)
            usec = round(usec * 1e6)

            # write payload to Ethernet frame:
            tactile_pcap_host.write_packet(tactile_header, sec=sec, usec=usec)
            pkt_id_host = inc_pkt_id(pkt_id_host)

        else:
            print("ERROR")
            print(hx, pkt_id_glove, pkt_id_host)
            break

## Podcast/Speech (Opus Audio)

In [3]:
with open("/home/arch/podcast/podcast.json") as audio_json_raw:
    audio_json = load(audio_json_raw)
df_a = DataFrame.from_dict(audio_json['frames'])
df_a['pkt_size'] = to_numeric(df_a['pkt_size'])

In [10]:
def get_header(payload, pkt_id, vlan_prio, eth_src):
    return Ether(dst=ETH_DST,
                 src=eth_src
                 )/Dot1Q(prio=vlan_prio,
                         vlan=VLAN_ID,
                         type=VLAN_TYPE
                         )/IP(id=pkt_id,
                              version=4,
                              dst='10.0.0.2',
                              src='10.0.0.1')/UDP(sport=10001,
                                                  dport=10002)/payload


def inc_pkt_id(pkt_id):
    if pkt_id >= 65535:
        return 0
    else:
        return pkt_id + 1

In [12]:
PATH = "./"
ETH_DST = '1c:1b:0d:9a:70:8c'
VLAN_TYPE = 0x0800
VLAN_ID = 3
AUDIO_PATH = PATH + 'audio.pcap'
AUDIO_ETH_SRC = '00:00:00:00:00:02'
AUDIO_VLAN_PRIO = 5

audio_pcap = RawPcapWriter(open(AUDIO_PATH, 'wb'),
                           nano=False,
                           linktype=1,
                           sync=True,
                           append=True)

pkt_id = 0
for _, row in df_a.iterrows():
    time = row['best_effort_timestamp_time'].rsplit(".")
    sec = int(time[0])
    usec = int(time[1])
    size = row['pkt_size'] * '0'
    audio_header = get_header(payload=size,
                              pkt_id=pkt_id,
                              vlan_prio=AUDIO_VLAN_PRIO,
                              eth_src=AUDIO_ETH_SRC)
    audio_pcap.write_header(audio_header)
    audio_pcap.write_packet(audio_header,
                            sec=sec,
                            usec=usec)
    pkt_id = inc_pkt_id(pkt_id)

## Spot Data Streams

In [2]:
def get_header(payload, pkt_id, vlan_prio, eth_dst, eth_src):
    return Ether(dst=eth_dst,
                 src=eth_src
                 )/Dot1Q(prio=vlan_prio,
                         vlan=VLAN_ID,
                         type=VLAN_TYPE
                         )/IP(id=pkt_id,
                              version=4,
                              dst='10.0.0.2',
                              src='10.0.0.1')/UDP(sport=10001,
                                                  dport=10002)/payload


def inc_pkt_id(pkt_id):
    if pkt_id >= 65535:
        return 0
    else:
        return pkt_id + 1

In [7]:
PATH = "./"
ETH_DST = '1c:1b:0d:9a:70:8c'
ETH_SRC = "00:00:00:00:00:03"
VLAN_TYPE = 0x0800
VLAN_ID = 3
VLAN_PRIO = 4
#WASD=6
#other=4

LEN_ETHERNET_HEADER = 14
LEN_VLAN_HEADER = 4
LEN_IP_HEADER = 20
LEN_UDP_HEADER = 8
LEN_HEADER = LEN_ETHERNET_HEADER + LEN_VLAN_HEADER + LEN_IP_HEADER + LEN_UDP_HEADER
LEN_FRAME_MIN = 64
FULL_PAYLOAD = 1518 - LEN_HEADER
FULL_PAYLOAD_ZEROS = FULL_PAYLOAD * '0'

STREAM = "RAW"
#wasd
#RAW
#JPEG

PCAP_PATH_HOST = "{}spot_{}{}.pcap".format(PATH, STREAM, "_host")
PCAP_PATH_SPOT = "{}spot_{}{}.pcap".format(PATH, STREAM, "_spot")

pcap_host = Reader(open("/home/arch/testbed_src/src/pcap_writer/spot_pcap/spot_{}_host-to-spot.pcap".format(STREAM), "rb"))
pcap_spot = Reader(open("/home/arch/testbed_src/src/pcap_writer/spot_pcap/spot_{}_spot-to-host.pcap".format(STREAM), "rb"))
pcaps = [pcap_host, pcap_spot]
rpw_host = RawPcapWriter(open(PCAP_PATH_HOST, 'wb'),
                         nano=False,
                         linktype=1,
                         sync=True,
                         append=True)
rpw_spot = RawPcapWriter(open(PCAP_PATH_SPOT, 'wb'),
                         nano=False,
                         linktype=1,
                         sync=True,
                         append=True)
pkt_id_host = 0
pkt_id_spot = 0
TIME_REF_HOST = None
TIME_REF_SPOT = None

for pcap in pcaps:

    ts_last_spot = 0
    ts_last_host = 0

    for ts, buf in pcap:

        hx = buf.hex()
        size = len(bytes.fromhex(hx[132:]))

        if pcap == pcap_spot:
            # set time reference:
            if not TIME_REF_SPOT:
                TIME_REF_SPOT = ts

            # set correct time:
            ts_new = ts - TIME_REF_SPOT

            # workaround for negative delta T: use last _ts_ + 1ns
            if Decimal(ts_new) - ts_last_spot <= 0:
                ts_new = round(ts_last_spot + Decimal(1e-6), 6)
            ts_last_spot = Decimal(ts_new)
            usec, sec = modf(ts_new)
            sec = int(sec)
            usec = round(usec * 1e6)

            full, part = divmod(size, FULL_PAYLOAD)
            for _ in range(0, full):
                spot_header = get_header(payload=FULL_PAYLOAD_ZEROS,
                                         pkt_id=pkt_id_spot,
                                         vlan_prio=VLAN_PRIO,
                                         eth_dst=ETH_DST,
                                         eth_src=ETH_SRC)
                rpw_spot.write_header(spot_header)
                rpw_spot.write_packet(spot_header,
                                      sec=sec,
                                      usec=usec)
                usec += 1
                ts_last_spot += Decimal(1e-6)
                pkt_id_spot = inc_pkt_id(pkt_id_spot)

            if part < (LEN_FRAME_MIN - LEN_HEADER):
                part = (LEN_FRAME_MIN - LEN_HEADER) * '0'
            else:
                part = part * '0'
            spot_header = get_header(payload=part,
                                     pkt_id=pkt_id_spot,
                                     vlan_prio=VLAN_PRIO,
                                     eth_dst=ETH_DST,
                                     eth_src=ETH_SRC)
            rpw_spot.write_header(spot_header)
            rpw_spot.write_packet(spot_header,
                                  sec=sec,
                                  usec=usec)
            pkt_id_spot = inc_pkt_id(pkt_id_spot)

        elif pcap == pcap_host:
            # set time reference:
            if not TIME_REF_HOST:
                TIME_REF_HOST = ts

            if size < (LEN_FRAME_MIN - LEN_HEADER):
                payload = (LEN_FRAME_MIN - LEN_HEADER) * '0'
            else:
                payload = size * '0'

            # create and write header for Ethernet frame:
            spot_header = get_header(payload=payload,
                                     pkt_id=pkt_id_host,
                                     vlan_prio=VLAN_PRIO,
                                     eth_dst=ETH_SRC,
                                     eth_src=ETH_DST)
            rpw_host.write_header(spot_header)

            # set correct time:
            ts_new = ts - TIME_REF_HOST

            # workaround for negative delta T: use last _ts_ + 1ns
            if ts_new - ts_last_host < 0:
                ts_new = round(ts_last_host + Decimal(1e-6), 6)
            ts_last_host = ts_new

            usec, sec = modf(ts_new)
            sec = int(sec)
            usec = round(usec * 1e6)

            # write payload to Ethernet frame:
            rpw_host.write_packet(spot_header,
                                  sec=sec,
                                  usec=usec)
            pkt_id_host = inc_pkt_id(pkt_id_host)